In [13]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [37]:
#Reading the datasets 
busstops = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/busstops_norway.csv')
grunnkrets_age = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_age_distribution.csv')
grunnkrets_household = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_households_num_persons.csv')
grunnkrets_income = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_income_households.csv')
grunnkrets_norway = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_norway_stripped.csv')
plaace_hierarchy = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/plaace_hierarchy.csv')
sample_submission = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/sample_submission.csv')
stores_extra = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_extra.csv')
stores_test = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_test.csv')
stores_train = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_train.csv')
simens_df = pd.read_csv("C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/notebooks/simen/simens_dataframe-1.csv")

# Issue 1: Population Feature 

Definition
Population number of a given geography

Datasets: Age

Todos: 
- Feature name: Population_in_GK
- For each grunnkrets_ID we must sum the amount of the row from age0 to age90 to calculate the total number of people in that grunnkrets
- NB! Check for duplicate rows

In [4]:
grunnkrets_age
grunnkrets_norway[grunnkrets_norway["year"] == 2016]

pop_df = grunnkrets_age.merge(grunnkrets_norway, how="left", on=("grunnkrets_id", "year"))
pop_df1 = pop_df[pop_df["year"] == 2016]
pop_df1


,grunnkrets_id,year,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_7,...,age_86,age_87,age_88,age_89,age_90,grunnkrets_name,district_name,municipality_name,geometry,area_km2
1,16013117,2016,10,10,10,10,10,10,9,9,...,0,0,0,0,0,Studentbyen,Berg-Tyholt,Trondheim,"POLYGON((10.4294131958998 63.4097004187783, 10...",0.080968
3,16011203,2016,2,2,2,2,2,3,3,3,...,1,1,0,0,0,Midtbyen 3,Midtbyen,Trondheim,"POLYGON((10.38529386234 63.4322940215389, 10.3...",0.095402
4,3011601,2016,7,7,7,7,6,6,5,4,...,0,0,0,0,0,Sagene rode 1,Sagene,Oslo,"POLYGON((10.7497785955323 59.9352432580208, 10...",0.033981
8,3010807,2016,3,3,3,3,3,3,2,2,...,0,0,0,0,0,Homansbyen rode 7,Homansbyen,Oslo,"POLYGON((10.7265000531186 59.9226629704886, 10...",0.053471
11,3012307,2016,4,4,4,4,4,4,4,4,...,0,0,0,0,0,Grünerløkka rode 7,Grünerløkka,Oslo,"POLYGON((10.7632427269883 59.920010687717, 10....",0.021916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22610,9380111,2016,0,0,0,0,0,0,0,1,...,1,1,0,0,0,Sandnes/Åraksbø,Bygland,Bygland,MULTIPOLYGON(((7.7546231430319 58.924594961388...,1.661184
22612,9400101,2016,0,0,0,1,1,1,1,1,...,0,0,0,0,0,Treungen,Valle,Valle,MULTIPOLYGON(((7.54074519425539 59.03546618966...,1.289755
22615,9400104,2016,1,1,1,1,1,1,0,0,...,0,0,0,0,0,"Gardane-Brokke,Haugeland, Berg",Valle,Valle,MULTIPOLYGON(((7.5084713600169 59.102764338814...,0.873378
22618,9400108,2016,0,1,1,0,0,0,1,1,...,1,0,0,0,0,Homme og Tveitebø,Valle,Valle,"POLYGON((7.54403767851663 59.2320152839935, 7....",2.011993


In [5]:
#This function calculates the population for each grunnkrets
#Returns a df with grunnkretsID in the first column and population_count in the second column

def population(dataset_age): 
    age_df = dataset_age[(dataset_age["year"] == 2016)]
    population = age_df.drop(["grunnkrets_id", "year"], axis=1).sum(axis=1)
    age_df["population_count"] = population 
    return age_df[["grunnkrets_id", "population_count"]]

population(grunnkrets_age)



<ipython-input-5-09da54aaad67>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population


,grunnkrets_id,population_count
1,16013117,944
3,16011203,880
4,3011601,645
8,3010807,528
11,3012307,876
...,...,...
22610,9380111,81
22612,9400101,78
22615,9400104,84
22618,9400108,151


In [6]:
#This function calculates the population in a district or municipality, by setting grouping_elemnt either to the district_name or municipality_name

def population_grouped(data_age, data_geography, grouping_element): 
    age_df = population(data_age)
    geography_df = data_geography[data_geography["year"] == 2016]
    population_df = age_df.merge(geography_df, how="left", on ="grunnkrets_id")
    grouped_df = population_df.groupby([grouping_element], as_index = False)["population_count"].sum()
    return grouped_df

population_grouped(grunnkrets_age, grunnkrets_norway, grouping_element="district_name")



<ipython-input-5-09da54aaad67>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population


,district_name,population_count
0,Agdenes,1683
1,Alfaset,18998
2,Algarheim,5780
3,Alstad,6400
4,Alstahaug/Tjøtta,817
...,...,...
1477,Øymark,2587
1478,Øyrekken,138
1479,Øystese,2719
1480,Øystre Slidre nord,1569


# Issue 2 Population Density Feature 
Definition
Population density in a given geography

Data Sources

- Population
- Grunnkrets (area)

Feature name: population_density_grunnkrets, population_density_district, population_density_kommune

Note:
Using population we get from population feature, divide with the area of grunnkrets, district and kommune. We get area of district and kommune by adding up the areas of grunnkrets with the same district. Same logic for district to kommune.

In [7]:
#This function calculates the density (population/area_km2) for the chosen grouping_element 

def population_density (age_df, geo_df, grouping_element): 
    age_data = population(age_df)
    geo_df = geo_df[geo_df["year"] == 2016]
    combined_df = age_data.merge(geo_df, how = "left", on = "grunnkrets_id")
    density_df = combined_df.groupby([grouping_element], as_index = False)[["population_count", "area_km2"]].sum()
    density_df["density"] = density_df["population_count"] / density_df["area_km2"]
    return density_df

population_density(grunnkrets_age, grunnkrets_norway, grouping_element="municipality_name")




<ipython-input-5-09da54aaad67>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population


,municipality_name,population_count,area_km2,Density
0,Agdenes,1683,24.198368,69.550145
1,Alstahaug,7189,34.378517,209.113153
2,Alta,19443,44.021857,441.666961
3,Alvdal,2202,30.079660,73.205615
4,Andebu,5837,37.999336,153.607948
...,...,...,...,...
417,Østre Toten,14692,151.675968,96.864389
418,Øvre Eiker,18002,79.545717,226.310112
419,Øyer,4989,38.138484,130.812750
420,Øygarden,4800,10.215312,469.882842


# Issue 9 Is_mall Feature 

Description
Boolean value of whether a store is a part of a mall

Details
Might be helpful to do some data cleaning. I.e. if coordinates of stores are in the same location of a mall, but the mall feature is missing -> impude.

Datasets

- Stores_train (primary)
- Stores_extra (secondary)

Name feature: Is_mall

-  Need to check if the stores in stores_train have missing value in column MallName since they are not in a Mall or that they have missing value but is actually a part of a Mall.
1. 
- Check the stores_train dataset for GK id with NaN in mall-coloum
- Check for corresponding GK id in stores_extra
- If mallName exist in stores_extra at the corresponding Long Lat -> use the fillna() function to fill the missing value in stores_train
2. 
- Iterate through stores_train column MallName
- If value = NaN => Is_mall = False
- Else -> Is_mall = True

In [8]:
#This function checks wether or not a store is part of a mall or not 

def is_mall(stores_df): 
    df = stores_df.copy()
    df["is_mall"] = df["mall_name"].notna()
    return df[["store_id", "mall_name", "is_mall"]]

    
is_mall(stores_train)







,store_id,mall_name,is_mall
0,983540538-974187930-44774,Magasinet Drammen,True
1,987074191-973117734-44755,NaN,False
2,984890265-981157303-64491,Kuben Hønefoss,True
3,914057442-992924179-126912,Glasshuspassasjen,True
4,913018583-913063538-668469,Tillertorget,True
...,...,...,...
12854,915789943-915806929-781991,NaN,False
12855,917921733-917982368-868081,CC Gjøvik,True
12856,911721961-911764474-496764,Strømmen Storsenter,True
12857,914337046-914343372-721294,Bystasjonen,True


In [9]:

stores_extra.head()

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name
0,911669196-973140302-27020,2016,BURGER KING STRANDGATEN,1.1.1.0,Hamburger restaurants,12010115,STRANDGATEN 5,60.393979,5.323851,BURGER KING,NaN
1,913155726-992998792-5832,2016,BURGER KING JESSHEIM STORSENTER,1.1.1.0,Hamburger restaurants,2350205,STORGATA 6,60.142760,11.171834,BURGER KING,Jessheim Storsenter
2,988252905-981108604-47887,2016,BURGER KING LANGNES,1.1.1.0,Hamburger restaurants,19020419,HULDERVEGEN 6,69.671483,18.920483,BURGER KING,NaN
3,995194546-891354622-45262,2016,BURGER KING MAGNETEN LEVANGER,1.1.1.0,Hamburger restaurants,17190701,NaN,63.732791,11.281785,BURGER KING,Magneten
4,881339692-979954964-3203,2016,BURGER KING ULLEVAAL STADION,1.1.1.0,Hamburger restaurants,3014511,SOGNSVEIEN 75,59.948558,10.732823,BURGER KING,Amfi Ullevaal


# Issue 10 Is_chain Feature 

Definition
A boolean value of whether a store is part of a chain.

Data Sources

- Stores



In [25]:
#This function checks wether or not a store is part of a chain or not 

def is_chain(stores_df): 
    df = stores_df.copy()
    df["is_chain"] = df["chain_name"].notna()
    return df[["store_id" , "chain_name", "is_chain"]]

is_chain(stores_train)



,store_id,chain_name,is_chain
0,983540538-974187930-44774,MCDONALDS,True
1,987074191-973117734-44755,MCDONALDS,True
2,984890265-981157303-64491,BURGER KING,True
3,914057442-992924179-126912,BURGER KING,True
4,913018583-913063538-668469,BURGER KING,True
...,...,...,...
12854,915789943-915806929-781991,NaN,False
12855,917921733-917982368-868081,GULATING GRUPPEN,True
12856,911721961-911764474-496764,GULATING GRUPPEN,True
12857,914337046-914343372-721294,NaN,False


# Issue 18 Population per store 

Definition
Population divided by the number of stores in a given geography.

Data Sources

- Stores
- Plaace
- Grunnkrets Norway

Potential breakdowns

- Breakdown by store type.

From the population feature, divide the number of stores in a given geography. An extension to this could be to do the breakdown by the store type label at different granularities from the place dataset.

In [36]:
#This function calculates the population count per number of stores in a geographic region 

def population_per_store(age_df, geo_df, stores_df, grouping_element):
    new_geo_df = geo_df[geo_df["year"] == 2016]
    pop_gk = population(age_df)
    pop_df = population_grouped(age_df, geo_df, grouping_element)
    combined_df = pop_gk.merge(stores_df, how = "left", on = "grunnkrets_id").merge(new_geo_df, how = "left", on = "grunnkrets_id")
    grouped_df = combined_df.groupby([grouping_element], as_index = False)["store_id"].count()
    pop_per_store_df = grouped_df.merge(pop_df, how = "inner", on = grouping_element)
    pop_per_store_df["population_per_num_stores"] = pop_per_store_df["population_count"] / pop_per_store_df["store_id"]
    pop_per_store_df.rename(columns = {"store_id": "num_stores"}, inplace = True)
    new_pop_per_store_df = pop_per_store_df.replace([np.inf, -np.inf], 0)
    return new_pop_per_store_df

population_per_store(grunnkrets_age, grunnkrets_norway, stores_train, grouping_element="district_name")


<ipython-input-5-09da54aaad67>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population
<ipython-input-5-09da54aaad67>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population


,district_name,num_stores,population_count,population_per_num_stores
0,Agdenes,1,1683,1683.000000
1,Alfaset,16,18998,1187.375000
2,Algarheim,0,5780,0.000000
3,Alstad,5,6400,1280.000000
4,Alstahaug/Tjøtta,1,817,817.000000
...,...,...,...,...
1477,Øymark,5,2587,517.400000
1478,Øyrekken,0,138,0.000000
1479,Øystese,10,2719,271.900000
1480,Øystre Slidre nord,17,1569,92.294118


# Issue Age distrubution Feature 
Definition
Distribution of age in a given geography.

Details
A table with a given geography as primary key and the age distribution bucketed as individual columns. E.g. 0-6 years, 6-12 years, etc. and the associated percentages.

Data Sources

- Age
- Grunnkrets

Group the age set by the following:
- Kids = 0-6 years
- Kids+ = 7-12 years
- Youths = 13-17 years
- YouthAdult = 18-25 years
- Adults = 26-35 years
- Adults+ = 36-60 years
- Pensinors = 60+ years

NB! The grouping can be changes based on the dataset, feel free to edit as needed.

Percentages could be calculated by: (number of persons in age group x in GK y) / (total amount in GK y)

In [19]:
#This function groups the age distrubution (0-90) into 7 buckets with and returns a table which represents the presentages each of these 
# buckets corresponds to compared with the total amount of people living in the given geographic region 

def age_distrubution(grunnkrets_age_df, geographic_df, grouping_element): 
    age_df = grunnkrets_age_df[grunnkrets_age_df["year"] == 2016]
    age_df1 = age_df.drop(["year"], axis = 1)
    age_df1["num_kids"] = age_df1.iloc[:, 1:8].sum(axis=1)   
    age_df1["num_kids+"] = age_df1.iloc[:, 8:14].sum(axis=1)
    age_df1["num_youths"] = age_df1.iloc[:, 14: 19].sum(axis=1)
    age_df1["num_youthAdult"] = age_df1.iloc[:, 19:27].sum(axis=1)
    age_df1["num_adult"] = age_df1.iloc[:, 27:37].sum(axis=1)
    age_df1["num_adults+"] = age_df1.iloc[:, 37:62].sum(axis=1)
    age_df1["num_pensinors"] = age_df1.iloc[:, 62:92].sum(axis=1)

    age_df2 = age_df1[["grunnkrets_id", "num_kids", "num_kids+", "num_youths", "num_youthAdult", "num_adult", "num_adults+", "num_pensinors"]]

    pop_df = population(grunnkrets_age_df)
    geo_df = geographic_df[geographic_df["year"] == 2016]
    new_geo_df = geo_df.drop(["geometry", "area_km2", "year"], axis = 1)
    combined_df = age_df2.merge(pop_df, how="inner", on = "grunnkrets_id").merge(new_geo_df, how = "inner", on = "grunnkrets_id")
    list_columns = ["num_kids", "num_kids+", "num_youths", "num_youthAdult", "num_adult", "num_adults+", "num_pensinors"]
    combined_df2 = combined_df.groupby([grouping_element], as_index=False)[list_columns].sum()

    pop_gk = population_grouped(grunnkrets_age_df, geographic_df, grouping_element)
    new_df = combined_df2.merge(pop_gk, how = "inner", on = grouping_element)

    new_df["kids_%"] = new_df["num_kids"] / new_df["population_count"]
    new_df["kids+_%"] = new_df["num_kids+"] / new_df["population_count"]
    new_df["youths_%"] = new_df["num_youths"] / new_df["population_count"]
    new_df["youthAdult_%"] = new_df["num_youthAdult"] / new_df["population_count"]
    new_df["adult_%"] = new_df["num_adult"] / new_df["population_count"]
    new_df["adults+_%"] = new_df["num_adults+"] / new_df["population_count"]
    new_df["pensinors_%"] = new_df["num_pensinors"] / new_df["population_count"]

    age_dist_df = new_df.drop(["population_count"], axis = 1)
    #if (grouping_element == "grunnkrets_id"): 
        #return new_df[["grunnkrets_id", "kids_%", "kids+_%", "youths_%", "youthAdult_%", "adult_%", "adults+_%", "pensinors_%" ]]
    #else: 
        #return new_df[[grouping_element, "kids_%", "kids+_%", "youths_%", "youthAdult_%", "adult_%", "adults+_%", "pensinors_%" ]]

    return age_dist_df
df = age_distrubution(grunnkrets_age, grunnkrets_norway,grouping_element="grunnkrets_id")

df


<ipython-input-5-09da54aaad67>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population
<ipython-input-5-09da54aaad67>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["population_count"] = population


,grunnkrets_id,num_kids,num_kids+,num_youths,num_youthAdult,num_adult,num_adults+,num_pensinors,kids_%,kids+_%,youths_%,youthAdult_%,adult_%,adults+_%,pensinors_%
0,1010102,22,30,34,59,56,171,256,0.035032,0.047771,0.054140,0.093949,0.089172,0.272293,0.407643
1,1010103,102,93,88,130,190,373,261,0.082458,0.075182,0.071140,0.105093,0.153597,0.301536,0.210994
2,1010104,55,40,35,79,97,199,201,0.077904,0.056657,0.049575,0.111898,0.137394,0.281870,0.284703
3,1010105,138,101,85,298,310,647,769,0.058773,0.043015,0.036201,0.126917,0.132027,0.275554,0.327513
4,1010107,42,38,28,124,109,200,178,0.058414,0.052851,0.038943,0.172462,0.151599,0.278164,0.247566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11299,20300501,4,6,10,3,17,37,19,0.041667,0.062500,0.104167,0.031250,0.177083,0.385417,0.197917
11300,20300502,7,9,10,10,15,41,25,0.059829,0.076923,0.085470,0.085470,0.128205,0.350427,0.213675
11301,20300503,28,18,19,23,30,81,42,0.116183,0.074689,0.078838,0.095436,0.124481,0.336100,0.174274
11302,20300504,7,7,13,23,15,61,37,0.042945,0.042945,0.079755,0.141104,0.092025,0.374233,0.226994


# Issue 14 Household type distrubution 

Definition

A feature for the distribution of household types in a given geography. This feature will have a similar structure to the age distribution feature.

Data Sources

- Households
- GK

In [ ]:
def household_type_distrubution(geographic_df, household_df, grouping_element): 
  house_df = household_df[household_df["year"] == 2016]
  geo_df = geographic_df[geographic_df["year"] == 2016]
  combined_df = geo_df.merge(house_df, how ="inner", on="grunnkrets_id")
  
  list_columns = ["couple_children_0_to_5_years", "couple_children_18_or_above", "couple_children_6_to_17_years", 
  "couple_without_children", "single_parent_children_0_to_5_years", "single_parent_children_18_or_above", 
  "single_parent_children_6_to_17_years", "singles" ]

  grouped_df = combined_df.groupby([grouping_element], as_index=False)[list_columns].sum()
  grouped_df["tot_pop_count"] = grouped_df.iloc[:, 1:].sum(axis=1)
  
  grouped_df["%_dist_of_couple_children_0_to_5_years"] = grouped_df["couple_children_0_to_5_years"] / grouped_df["tot_pop_count"]
  grouped_df["%_dist_of_couple_children_18_or_above"] = grouped_df["couple_children_18_or_above"] / grouped_df["tot_pop_count"]
  grouped_df["%_dist_of_couple_children_6_to_17_years"] = grouped_df["couple_children_6_to_17_years"] / grouped_df["tot_pop_count"]
  grouped_df["%_dist_of_couple_without_children"] = grouped_df["couple_without_children"] / grouped_df["tot_pop_count"]
  grouped_df["%_dist_of_single_parent_children_0_to_5_years"] = grouped_df["single_parent_children_0_to_5_years"] / grouped_df["tot_pop_count"]
  grouped_df["%_dist_of_single_parent_children_18_or_above"] = grouped_df["single_parent_children_18_or_above"] / grouped_df["tot_pop_count"]
  grouped_df["%_dist_of_single_parent_children_6_to_17_years"] = grouped_df["single_parent_children_6_to_17_years"] / grouped_df["tot_pop_count"]
  grouped_df["%_dist_of_singles"] = grouped_df["singles"] / grouped_df["tot_pop_count"]

  returned_df = grouped_df.drop(["tot_pop_count"], axis = 1)
  return returned_df


household_type_distrubution(grunnkrets_norway, grunnkrets_household, grouping_element="grunnkrets_name")



,grunnkrets_name,couple_children_0_to_5_years,couple_children_18_or_above,couple_children_6_to_17_years,couple_without_children,single_parent_children_0_to_5_years,single_parent_children_18_or_above,single_parent_children_6_to_17_years,singles,%_dist_of_couple_children_0_to_5_years,%_dist_of_couple_children_18_or_above,%_dist_of_couple_children_6_to_17_years,%_dist_of_couple_without_children,%_dist_of_single_parent_children_0_to_5_years,%_dist_of_single_parent_children_18_or_above,%_dist_of_single_parent_children_6_to_17_years,%_dist_of_singles
0,1. og 2. Vannsområdet,32,26,39,62,0,0,29,82,0.118519,0.096296,0.144444,0.229630,0.000000,0.000000,0.107407,0.303704
1,A. G. Johnsons vei,56,26,104,78,0,6,20,35,0.172308,0.080000,0.320000,0.240000,0.000000,0.018462,0.061538,0.107692
2,A.V.L.,47,12,129,44,12,5,10,40,0.157191,0.040134,0.431438,0.147157,0.040134,0.016722,0.033445,0.133779
3,Aa,37,41,69,82,13,17,4,70,0.111111,0.123123,0.207207,0.246246,0.039039,0.051051,0.012012,0.210210
4,Aaen vest 1,14,0,35,20,0,4,0,12,0.164706,0.000000,0.411765,0.235294,0.000000,0.047059,0.000000,0.141176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11396,Øyslebø,72,33,99,66,8,8,12,61,0.200557,0.091922,0.275766,0.183844,0.022284,0.022284,0.033426,0.169916
11397,Øysletta,52,24,74,48,7,4,0,37,0.211382,0.097561,0.300813,0.195122,0.028455,0.016260,0.000000,0.150407
11398,Øystese,157,122,182,174,14,15,18,194,0.179224,0.139269,0.207763,0.198630,0.015982,0.017123,0.020548,0.221461
11399,Øyum,53,32,42,46,0,10,12,35,0.230435,0.139130,0.182609,0.200000,0.000000,0.043478,0.052174,0.152174
